In [1]:
import numpy as np 
from scipy import optimize
from scipy.optimize import NonlinearConstraint
from scipy.optimize import minimize
import os

from tqdm import * 
import pickle 

import matplotlib.pyplot as plt 
%matplotlib inline 

In [2]:
def func(x): 
    
    return np.sum( np.sin(x) ) + np.exp((x[0]-1) * (x[1]+2))  

In [3]:
def sample_stiefel(n,k): 
    
    U = np.random.normal( 0, 1, (n, k) ) 
    
    L,S,R = np.linalg.svd(np.matmul(U.T, U))
    
    U_ = np.matmul( np.matmul(L, np.diag( S**(-1./2) ) ), R )
        
    return np.matmul(U,U_)


# def sample_stiefel(n,k): 
    
#     A = np.random.normal( 0, 1, (n, k) ) 
    
#     def normalize(v): 
#         return v / np.sqrt(v.dot(v)) 

#     n = A.shape[1] 

#     A[:, 0] = normalize(A[:, 0])  

#     for i in range(1, n): 
#         Ai = A[:, i] 
#         for j in range(0, i): 
#             Aj = A[:, j] 
#             t = Ai.dot(Aj) 
#             Ai = Ai - t * Aj 
#         A[:, i] = normalize(Ai) 
        
#     return A 

# our method, Stiefel's sampling

In [4]:
def stiefel( x, delta, k, n = 100 ): 


    V = sample_stiefel(n,k) 
    W = sample_stiefel(n,k) 
    
    H = 0
    for i in range(k): 
        for j in range(k): 
            
            H = H + n**2/delta**2/8 * ( func( x + delta * V[:,i] + delta * W[:,j]) \
                            - func( x - delta * V[:,i] + delta * W[:,j]) \
                            - func( x + delta * V[:,i] - delta * W[:,j]) \
                            + func( x - delta * V[:,i] - delta * W[:,j])) \
                    * ( np.outer(V[:,i], W[:,j]) + np.outer( W[:,j], V[:,i]) ) 
            
    H = H / k**2 
    
    return H 

# spherical method

In [5]:
def spherical( x, delta, k, n = 100 ): 
   
    H = 0
    
    for _ in range(k):
        for _ in range(k):
            V = sample_stiefel(n,1)[:,0] 
            W = sample_stiefel(n,1)[:,0] 
            
            H = H + n**2/delta**2/8 * ( func( x + delta * V + delta * W) \
                            - func( x - delta * V + delta * W) \
                            - func( x + delta * V - delta * W) \
                            + func( x - delta * V - delta * W)) \
                    * ( np.outer(V, W) + np.outer( W, V) ) 

    H = H / k**2 
    
    return H 

# gaussian method

In [6]:
def gaussian( x, delta, k, n = 100): 
    
    H = 0 
    
    for _ in range(k**2): 
        
        V = np.random.normal(0,1,n) 
            
        H = H + n/2/delta**2 * ( func( x + delta * V / np.sqrt(n) ) \
                            - 2*func( x ) \
                            + func( x - delta * V / np.sqrt(n) ) ) \
                    * ( np.outer(V, V) - np.identity(n) ) 

    H = H / k**2 
    
    return H 

# entry-wise

In [7]:
def entry( x, delta, n = 100 ): 
    
    H = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            ei = np.zeros(n)
            ei[i] = 1
            ej = np.zeros(n)
            ej[j] = 1
            
            H[i,j] = 1./4/delta**2 * ( func( x + delta * ei + delta * ej ) \
                            - func( x + delta * ei - delta * ej ) - func( x - delta * ei + delta * ej ) \
                            + func( x - delta * ei - delta * ej ) ) 
    
    return H 

In [8]:
def truth(x, n = 100): 
    
    H = np.diag( -np.sin(x) ) 
    C = np.exp((x[0] - 1)*(x[1] + 2)) 
    H[0,0] = H[0,0] + (x[1] + 2)**2 * C 
    H[1,1] = H[1,1] + (x[0] - 1)**2 * C 
    
    H[0,1] = C + (x[0] - 1)* (x[1] + 2) * C
    H[1,0] = C + (x[0] - 1)* (x[1] + 2) * C
    
    return H

# following block for getting estimations and computing errrors

In [ ]:
n = 100 

xs = [np.array([0]*n), np.array([np.pi/4.]*n), -np.array([np.pi/2.]*n) ] 
locs = ['0', '0.25pi', 'n0.5pi'] 

deltas = [ 0.1 , 0.01, 0.001 ] 

ks = [ 20,40,60,80,100 ] 
# ks = [20]

reps = 10 

for i in range(len(xs)):
    
    x = xs[i]
    
    loc = locs[i]
    
    H = truth(x,n) 
    
    for delta in deltas: 
        
        for k in ks: 
            
            stiefel_errors = [] 
            spherical_errors = [] 
            gaussian_errors = [] 
                        
            for _ in range(reps):
                
                stiefel_H = stiefel( x, delta, k , n) 
                spherical_H = spherical( x, delta, k , n) 
                gaussian_H = gaussian( x, delta, k , n) 
                
                stiefel_errors.append( np.sqrt( np.sum( ( np.linalg.svd( stiefel_H - H )[1] )**2 ) ) )
                spherical_errors.append( np.sqrt( np.sum( (np.linalg.svd( spherical_H - H )[1] )**2 ) ) )
                gaussian_errors.append( np.sqrt( np.sum( (np.linalg.svd( gaussian_H - H )[1] )**2 ) ) )
                
#             print(k, np.mean( stiefel_errors ), np.mean( spherical_errors ), np.mean( gaussian_errors ) )
                
            pickle.dump( stiefel_errors, 
                        open('./raw_results/stiefel_hess_errors_x{0}_delta{1}_k{2}'.format(loc,delta,k), 'wb' ) ) 
            pickle.dump( spherical_errors, 
                        open('./raw_results/spherical_hess_errors_x{0}_delta{1}_k{2}'.format(loc,delta,k), 'wb' ) ) 
            pickle.dump( gaussian_errors, 
                        open('./raw_results/gaussian_hess_errors_x{0}_delta{1}_k{2}'.format(loc,delta,k), 'wb' ) ) 

# results of entry-wise estimator

In [23]:
n = 100

xs = [np.array([np.pi/2.]*n), np.array([np.pi/4.]*n) ] 

deltas = [ 0.1, 0.01, 0.001 ] 

reps = 10 

for i in range(len(xs)): 
    
    x = xs[i]
    
    H = truth(x, n)
    
    for delta in deltas: 
            
        entry_H = entry( x, delta, n ) 
        
        entry_wise_errors =  np.max( np.linalg.svd( entry_H - H )[1] ) 
#         np.linalg.norm( entry_H - H, ord = 2 )  
        
        print(i,delta,entry_wise_errors) 
                
                
#         pickle.dump( entry_wise_errors, 
#                 open('./raw_results/entry_wise_errors_x{0}_delta{1}'.format(i,delta), 'wb' ) ) 
                

0 0.1 4.400206672166374
0 0.01 0.04328676449516043
0 0.001 0.00043279415493315703
1 0.1 0.11649030818392277
1 0.01 0.001153473802530225
1 0.001 1.153206002120819e-05


In [37]:
n = 100

xs = [np.array([np.pi/2.]*n), np.array([np.pi/4.]*n) ] 

deltas = [ 0.1, 0.01, 0.001 ] 

reps = 10

for i in range(len(xs)): 
    
    x = xs[i]
    
    H = truth(x, n)
    
    for delta in deltas: 
        
        res = []
        
        for _ in range(reps): 
            
            stiefel_H = stiefel( x, delta, n, n )  

            stiefel_errors = np.max( np.linalg.svd( stiefel_H - H )[1] )
        
            res.append(stiefel_errors) 

#         stiefel_errors = pickle.load( 
            
#             open( './raw_results/stiefel_hess_errors_x{0}_delta{1}_k100'.format(i,delta) , 'rb') )
        
        print(i,delta, np.mean(res), np.std(res) ) 

0 0.1 0.1673588270866899 0.02046914059577767
0 0.01 0.0015556226556718575 8.285270278422251e-05
0 0.001 1.5284137412673218e-05 1.4054028109155922e-06
1 0.1 0.0037926729296291007 0.00041814693410627653
1 0.01 3.9098614124382936e-05 4.846701218944252e-06
1 0.001 3.9784717212638223e-07 4.259605873196608e-08
